# Regresión logística multiclase

La regresión logística es normalmente un modelo de clasificación binaria, es decir, distingue entre dos clases (por ejemplo: "spam" vs "no spam").
Pero si se tienen más de dos clases (por ejemplo, clasificar entre 3 marcas de vehículos), se necesita adaptar el modelo. Aquí es donde entra el enfoque One-vs-Rest:

*¿Cómo funciona One-vs-Rest?*
Suponiendo que se tienen tres clases: A, B y C.
Con One-vs-Rest vas a entrenar un clasificador binario por cada clase:
* Clasificador 1: ¿Es A o no? (A vs B+C)
* Clasificador 2: ¿Es B o no? (B vs A+C)
* Clasificador 3: ¿Es C o no? (C vs A+B)
Cada uno de estos clasificadores se entrena por separado usando regresión logística binaria. Luego, cuando se hace una predicción, cada uno da una probabilidad, y se elige la clase con la mayor probabilidad.

Inicialmente importaremos todas las librerias necesarias para la ejecución de nuestros procesos:

In [7]:
import pandas as pd

Ahora cargaremos los datasets que se utilizarán, estos estan en el siguiente orden de carpetas

```bash
|--datasources
   |--laptops
   |--parkinsons
```

In [10]:
laptops = pd.read_csv("./datasources/laptops/laptop_price.csv")